In [ ]:
%load_ext autoreload
%autoreload 2


import uproot
import awkward as ak

import matplotlib.pylab as plt
import numpy as np

import time

from hist import Hist

import babar_analysis_tools as bat

import pandas as pd

from analysis_variables import *

# Open the data file or files

In [ ]:
start = time.time()

# At Siena
topdir = '/mnt/qnap/babar_data/bnv_plambda'

# At Bellis' home
#topdir = '/home/bellis/babar_data/bnv_plambda'

# On Bellis' laptop
#topdir = './'

#filename = f'{topdir}/Background_SP_modes_Only_Run_1.parquet'
filename = f'{topdir}/Background_and_signal_SP_modes_Only_Run_1.parquet'

data = ak.from_parquet(filename)

print(f"Took {time.time() - start} s")


# Plots and tables for signal and blinding area

In [ ]:
from analysis_variables import *
region_definitions

In [ ]:
def table_from_df(df):
    output = df.to_latex(index=False,
                  float_format="{:.4f}".format,
    )  # converts dataframe into latex readable text
    full_table = "\\begin{table}\n" # initializes the table before the beginning of the tabular 
    full_table += "\\caption{This could be the caption}\n" 
    full_table += output #includes the converted dataframe in the table
    full_table += "\\end{table}" # ends the table, same purpose as begin{table} 
    return full_table #make sure to return the print() of the full_table, otherwise it'll be one big string that latex can't handle

In [ ]:
rd = region_definitions

dict_temp = {}
dict_temp['Region'] = ['Fitting', 'Blinding']
dict_temp['MES low'] = [rd['fitting MES'][0], rd['blinding MES'][0]]
dict_temp['MES high'] = [rd['fitting MES'][1], rd['blinding MES'][1]]

dict_temp['DeltaE low'] = [rd['fitting DeltaE'][0], rd['blinding DeltaE'][0]]
dict_temp['DeltaE high'] = [rd['fitting DeltaE'][1], rd['blinding DeltaE'][1]]


dftmp = pd.DataFrame.from_dict(dict_temp)

dftmp

In [ ]:
#table = table_from_df(dftmp)

header = ['Region', '$MES$ low', '$MES$ high', '$\Delta E$ low', '$\Delta E$ high']

output = dftmp.to_latex(index=False, header=header, 
              float_format="{:.2f}".format,
)  # converts dataframe into latex readable text

full_table = "\\begin{table}\n" # initializes the table before the beginning of the tabular 
full_table += "\\centering"
full_table += "\\caption{Definition of the fitting region and blinding region for this analysis.\\label{tab:def_regions}}\n" 
full_table += output #includes the converted dataframe in the table
full_table += "\\end{table}" # ends the table, same purpose as begin{table} 
#return full_table #make sure to return the print() of the full_table, otherwise it'll be one big string that latex can't handle

table = full_table


print(table)

outfilename = 'tables/table_def_regions.tex'
outfile = open(outfilename, 'w')
outfile.write(table)
outfile.close()
